# Rate law notebook

By **Peter Kraus and Paolo Raiteri**, January 2021

## Instructions:

- Adjust the concentration of `A` and `B` using the sliders below, 
- Click `Calculate` to obtain the reaction rate,
- Click `Print` to show table of previous results,
- Click `Download` to export the table as a csv file.

In [1]:
import ipywidgets as ipw
import json
import random
import pandas as pd
import os
import webbrowser

In [2]:
# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["test"]


In [3]:
# delete existing result file and setup rng
if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
    os.remove(os.path.join(os.getcwd(), "results.csv"))
random.seed(params["error"].get("seed", 0))

In [4]:
out_R = ipw.Label(value="Click Calculate.")
out_P = ipw.Output()

def calc(btn):
    R = params["k"] * conc_A.value ** params["x"] * conc_B.value ** params["y"]
    dR = R * params["error"].get("random", 0) * (0.5 - random.random()) + R * params["error"].get("systematic", 0)
    out_R.value = f"{R + dR:5.3E}"
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=["[A] (mol/l)", "[B] (mol/l)", "R (mol/l·s)"])
    res = res.append({"[A] (mol/l)": conc_A.value, "[B] (mol/l)": conc_B.value, "R (mol/l·s)": R + dR}, 
                     ignore_index=True)
    res.to_csv("results.csv", index=False)

def printres(btn):
    out_P.clear_output()
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=["[A] (mol/l)", "[B] (mol/l)", "R (mol/l·s)"])
    with out_P:
        display(res.tail())

def download(btn):
    url = "./results.csv"
    webbrowser.open(url)
        
conc_A = ipw.FloatLogSlider(value=params["[A]"], min=-2, max=1)
conc_B = ipw.FloatLogSlider(value=params["[B]"], min=-2, max=1)
btn_calc = ipw.Button(description="Calculate", layout=ipw.Layout(width="90px"))
btn_calc.on_click(calc)
btn_print = ipw.Button(description="Print", layout=ipw.Layout(width="50px"))
btn_print.on_click(printres)
btn_down = ipw.Button(description="Download", layout=ipw.Layout(width="90px"))
btn_down.on_click(download)

rows = []
rows.append(ipw.HBox([ipw.Label(value="Concentration of A:", layout=ipw.Layout(width="120px")), 
                      conc_A, 
                      ipw.Label(value="mol/l")]))
rows.append(ipw.HBox([ipw.Label(value="Concentration of B:", layout=ipw.Layout(width="120px")), 
                      conc_B, 
                      ipw.Label(value="mol/l")]))
rows.append(ipw.HBox([ipw.Label(value="Reaction rate is:", layout=ipw.Layout(width="120px")), 
                      out_R,
                      ipw.Label(value="mol/l·s")]))
rows.append(ipw.HBox([btn_calc, btn_print, btn_down]))
rows.append(ipw.HBox([out_P]))
calc(btn_calc)
ipw.VBox(rows)

[Download CSV](./results.csv)